In [112]:
import numpy as np
import pandas as pd
import os
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime as dt

print(os.getcwd())
for dirname, _, filenames in os.walk('./数据'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

E:\Code\MyGitSpace\MachineLearning\MachineLearning(3.8.16)\23\E
./数据\~$表1-患者列表及临床信息.xlsx
./数据\~$表2-患者影像信息血肿及水肿的体积及位置.xlsx
./数据\~$表3-患者影像信息血肿及水肿的形状及灰度分布.xlsx
./数据\~$表4-答案文件.xlsx
./数据\~$附表1-检索表格-流水号vs时间.xlsx
./数据\表1-患者列表及临床信息.xlsx
./数据\表2-患者影像信息血肿及水肿的体积及位置.xlsx
./数据\表3-患者影像信息血肿及水肿的形状及灰度分布.xlsx
./数据\表4-答案文件.xlsx
./数据\附表1-检索表格-流水号vs时间.xlsx


In [ ]:
table1 = pd.read_excel('./数据\表1-患者列表及临床信息.xlsx')
display(table1.head(5))
table2 = pd.read_excel('./数据\表2-患者影像信息血肿及水肿的体积及位置.xlsx')
display(table2.head(5))
table3_ED = pd.read_excel('./数据\表3-患者影像信息血肿及水肿的形状及灰度分布.xlsx',sheet_name='ED')
display(table3_ED.head(5))
table3_Hemo = pd.read_excel('./数据\表3-患者影像信息血肿及水肿的形状及灰度分布.xlsx',sheet_name='Hemo')
display(table3_Hemo.head(5))
table4 = pd.read_excel('./数据\表4-答案文件.xlsx')
display(table4.head(5))
table5 = pd.read_excel('./数据\附表1-检索表格-流水号vs时间.xlsx')
display(table5.head(5))
clear_output()

In [ ]:
Problem1a = pd.DataFrame()
Problem1a['发病到首次影像检查时间间隔'] = table1[table1.index<100]['发病到首次影像检查时间间隔']
Problem1a['resT'] = [pd.Timedelta(hours=i) for i in (48 - table1[table1.index<100]['发病到首次影像检查时间间隔'])]
Problem1a = pd.concat([Problem1a,table5[table5.index<100]], axis=1)
Problem1a = pd.concat([Problem1a,table2.iloc[range(0,100),range(2,9*23+1,23)]], axis=1)
Problem1a['发病时间'] = [i - pd.Timedelta(hours=j) for i,j in zip(pd.to_datetime(Problem1a['入院首次检查时间点']),table1[table1.index<100]['发病到首次影像检查时间间隔'])]
Problem1a['isExpansion_count'] = 0
# display(Problem1a['resT'])
for index in range(1,9):
#     display((pd.to_datetime(Problem1a['随访{}时间点'.format(index)]) - pd.to_datetime(Problem1a['入院首次检查时间点'])))
#     display((pd.to_datetime(Problem1a['随访{}时间点'.format(index)]) - pd.to_datetime(Problem1a['入院首次检查时间点'])) < Problem1a['resT'])
#     display(abs(Problem1a['HM_volume.{}'.format(index)]-Problem1a['HM_volume']) >= 6000)
#     display((abs(Problem1a['HM_volume.{}'.format(index)]-Problem1a['HM_volume'])/Problem1a['HM_volume']) >= 0.33)
    Problem1a['deltaT{}'.format(index)] = (pd.to_datetime(Problem1a['随访{}时间点'.format(index)]) - pd.to_datetime(Problem1a['入院首次检查时间点'])) < Problem1a['resT']
    Problem1a['deltaVolume{}'.format(index)] = (Problem1a['HM_volume.{}'.format(index)]-Problem1a['HM_volume']).fillna(0)
    Problem1a['deltaRelativeVolume{}'.format(index)] = ((Problem1a['HM_volume.{}'.format(index)]-Problem1a['HM_volume'])/Problem1a['HM_volume']).fillna(0) 
    Problem1a['deltaV{}'.format(index)] = np.logical_or(Problem1a['deltaVolume{}'.format(index)]>=6000,Problem1a['deltaRelativeVolume{}'.format(index)]>=0.33)
    Problem1a['isExpansion{}'.format(index)] = np.logical_and(Problem1a['deltaT{}'.format(index)],Problem1a['deltaV{}'.format(index)])
    Problem1a['isExpansion_count'] = Problem1a['isExpansion_count'] + Problem1a['isExpansion{}'.format(index)]
    if index > 1:
        pass
Problem1a['isExpansion'] = [int(i) for i in Problem1a['isExpansion_count']>=1]
def calExpansionTime(Problem1a):
    for row in range(0,100):
        if Problem1a.loc[row,'isExpansion']==True:
            for index in range(1,9):
                if Problem1a.loc[row,'isExpansion{}'.format(index)]==True:
                    Problem1a.loc[row,'isExpansion_time'] = (Problem1a.loc[row,'随访{}时间点'.format(index)]-Problem1a.loc[row,'发病时间'])*24
                    break;
calExpansionTime(Problem1a)
Problem1a.to_excel('Problem1a.xlsx')
# Problem1a['isExpansion1'] = sum(Problem1a['isExpansion{}'.format(index)])
# display(Problem1a.loc[range(0,100),['deltaT{}'.format(index) for index in range(1,9)]].head(5))
# display(Problem1a.loc[range(0,100),['deltaVolume{}'.format(index) for index in range(1,9)]])
# display(Problem1a.loc[range(0,100),['deltaRelativeVolume{}'.format(index) for index in range(1,9)]])
# display(Problem1a.loc[range(0,100),['deltaV{}'.format(index) for index in range(1,9)]].head(5))
# display(Problem1a.loc[range(0,100),['isExpansion{}'.format(index) for index in range(1,9)]])
# display(Problem1a['isExpansion'])
# dt.datetime.strptime(Problem1a['随访{}时间点'.format(index)],"%Y/%m/%d %H-%M")

In [ ]:
Problem1b = table1
Problem1b = pd.concat([Problem1b,table2.iloc[:,range(2,24)]],axis=1)
def findRow(Problem1b,table,row):
    for row1,num in enumerate(table['流水号']):
        if num==Problem1b.loc[row,'入院首次影像检查流水号']:
            return table.iloc[row1,range(2,33)]
pdtemp = pd.DataFrame(columns=table3_ED.columns[2:])
pdtemp1 = pd.DataFrame(columns=table3_Hemo.columns[2:])
for row in range(0,160):
    pdtemp.loc[row] = findRow(Problem1b,table3_ED,row)
    pdtemp1.loc[row] = findRow(Problem1b,table3_Hemo,row)

pdtemp.columns = pdtemp.columns + '_ED'
pdtemp1.columns = pdtemp1.columns + '_Hemo'
Problem1b = pd.concat([Problem1b,pdtemp,pdtemp1],axis=1)
Problem1b['target'] = Problem1a['isExpansion']
Problem1b['性别']=Problem1b['性别'].map({'男':1,'女':0}).astype(np.int8)
Problem1b.to_excel('Problem1b.xlsx')
display(Problem1b.head(5))

In [ ]:
drop_columns = ['ID','90天mRS','数据集划分','入院首次影像检查流水号']
display(Problem1b.drop(columns=drop_columns).head(5))

In [ ]:
train_df = Problem1b.drop(columns=drop_columns).loc[range(0,100)]
test1_df = Problem1b.drop(columns=drop_columns).loc[range(100,130)]
test2_df = Problem1b.drop(columns=drop_columns).loc[range(130,160)]
display(train_df)
display(test1_df)
display(test2_df)

In [ ]:
train_missing = train_df.isnull().sum() * 100 / len(train_df)
train_missing = train_missing[train_missing != 0]
train_missing = pd.DataFrame({'missing percent': train_missing})
train_missing = train_missing.sort_values('missing percent', ascending=False)
display(train_missing.transpose().style.format(precision = 3))
for col in train_missing.index:
    print(col,train_df[col].unique())
    
test1_missing = test1_df.isnull().sum() * 100 / len(test1_df)
test1_missing = test1_missing[test1_missing != 0]
test1_missing = pd.DataFrame({'missing percent': test1_missing})
test1_missing = test1_missing.sort_values('missing percent', ascending=False)
display(test1_missing.transpose().style.format(precision = 3))
for col in test1_missing.index:
    print(col,test1_df[col].unique())
    
test2_missing = test2_df.isnull().sum() * 100 / len(test2_df)
test2_missing = test2_missing[test2_missing != 0]
test2_missing = pd.DataFrame({'missing percent': test2_missing})
test2_missing = test2_missing.sort_values('missing percent', ascending=False)
display(test2_missing.transpose().style.format(precision = 3))
for col in test2_missing.index:
    print(col,test2_df[col].unique())

In [ ]:
# Pipeline([
#     ('imputer', SimpleImputer(strategy="median")),
#     ('std_scaler', StandardScaler()),
# ])

In [102]:
train_df = Problem1b.drop(columns=drop_columns).loc[range(0,100)]
test1_df = Problem1b.drop(columns=drop_columns).loc[range(100,130)]
test2_df = Problem1b.drop(columns=drop_columns).loc[range(130,160)]
display(train_df)
display(test1_df)
display(test2_df)

,年龄,性别,脑出血前mRS评分,高血压病史,卒中病史,糖尿病史,房颤史,冠心病史,吸烟史,饮酒史,...,NCCT_original_firstorder_MeanAbsoluteDeviation,NCCT_original_firstorder_Mean,NCCT_original_firstorder_Median,NCCT_original_firstorder_Minimum,NCCT_original_firstorder_Range,NCCT_original_firstorder_RobustMeanAbsoluteDeviation,NCCT_original_firstorder_RootMeanSquared,NCCT_original_firstorder_Skewness,NCCT_original_firstorder_Uniformity,NCCT_original_firstorder_Variance
0,43,0,0,0,0,0,0,0,0,0,...,8.247517,53.100174,54.277975,2.630186,88.333203,5.421960,54.182743,-0.680312,0.156607,116.141179
1,58,1,0,1,0,0,0,0,0,0,...,5.767857,37.252825,37.518627,11.670399,54.126259,3.986374,37.958520,-0.176969,0.132046,53.076248
2,78,1,0,1,0,0,0,0,0,0,...,7.659346,35.163065,35.410096,-1.715890,70.139106,5.382112,36.444232,-0.057096,0.128060,91.740851
3,70,1,2,1,1,0,0,0,0,0,...,8.911068,30.191045,30.926762,-20.991359,92.048538,6.191501,32.204819,-0.297133,0.145292,125.651157
4,51,1,0,0,0,0,0,0,0,0,...,9.250453,45.985845,46.492200,-9.889141,96.821808,6.482624,47.428035,-0.227378,0.146488,134.720581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,93,0,0,0,0,0,0,0,0,0,...,7.118580,44.469975,45.627698,-1.353307,75.353316,4.764270,45.427431,-0.852965,0.155848,86.072788
96,83,0,0,1,0,0,0,0,0,0,...,8.006967,49.624525,49.646822,10.589739,69.684072,5.654763,50.616489,-0.047266,0.121547,99.435503
97,82,1,0,1,1,1,1,1,1,0,...,8.462658,35.911280,35.494566,-2.467873,98.855412,5.523299,37.597608,0.432499,0.166861,123.960058
98,53,1,0,1,0,0,0,0,0,0,...,6.006107,41.019959,41.559362,3.177012,66.837891,4.002934,41.752598,-0.318306,0.159415,60.642372


,年龄,性别,脑出血前mRS评分,高血压病史,卒中病史,糖尿病史,房颤史,冠心病史,吸烟史,饮酒史,...,NCCT_original_firstorder_MeanAbsoluteDeviation,NCCT_original_firstorder_Mean,NCCT_original_firstorder_Median,NCCT_original_firstorder_Minimum,NCCT_original_firstorder_Range,NCCT_original_firstorder_RobustMeanAbsoluteDeviation,NCCT_original_firstorder_RootMeanSquared,NCCT_original_firstorder_Skewness,NCCT_original_firstorder_Uniformity,NCCT_original_firstorder_Variance
100,77,0,0,1,1,1,0,0,0,0,...,5.758658,45.017123,44.964374,8.735740,65.603342,3.796703,45.649313,-0.164068,0.162024,57.318392
101,70,1,1,1,1,0,0,0,0,0,...,7.714488,51.272843,52.127157,25.131743,54.484164,5.294308,52.182772,-0.207116,0.103243,94.137273
102,64,1,3,1,1,0,0,0,0,0,...,6.858109,24.272313,24.692394,-0.279029,46.051473,4.856257,25.714474,-0.191929,0.094309,72.089008
103,42,1,0,1,0,0,0,0,1,1,...,7.614996,59.130507,59.234266,9.605064,94.574340,5.110966,59.953682,-0.202080,0.174804,98.027071
104,51,1,0,1,1,0,0,0,0,0,...,5.569341,45.674851,46.014476,23.120501,40.453686,3.806922,46.217790,-0.197376,0.105396,49.892106
105,75,0,0,1,0,1,0,0,0,0,...,5.738569,59.183892,59.599173,30.645243,52.026206,3.906218,59.636615,-0.350650,0.128969,53.792767
106,63,0,0,1,0,0,0,0,0,0,...,7.896143,50.860156,51.325924,-1.404012,93.529091,5.371222,51.860958,-0.365170,0.166789,102.803535
107,67,1,0,1,0,0,1,0,0,0,...,7.903705,43.421757,43.922336,-6.131279,86.096867,5.369641,44.593162,-0.307711,0.153881,103.101097
108,82,1,0,0,1,0,0,1,0,0,...,5.766524,39.873702,39.398558,21.213643,36.922799,3.929889,40.535875,0.213288,0.103184,53.245028
109,56,1,0,0,0,1,0,0,0,0,...,6.118011,52.510815,52.383389,21.071895,61.012355,4.180407,53.089692,0.028300,0.140441,61.129685


,年龄,性别,脑出血前mRS评分,高血压病史,卒中病史,糖尿病史,房颤史,冠心病史,吸烟史,饮酒史,...,NCCT_original_firstorder_MeanAbsoluteDeviation,NCCT_original_firstorder_Mean,NCCT_original_firstorder_Median,NCCT_original_firstorder_Minimum,NCCT_original_firstorder_Range,NCCT_original_firstorder_RobustMeanAbsoluteDeviation,NCCT_original_firstorder_RootMeanSquared,NCCT_original_firstorder_Skewness,NCCT_original_firstorder_Uniformity,NCCT_original_firstorder_Variance
130,78,1,0,1,0,0,0,0,0,0,...,7.659346,35.163065,35.410096,-1.715890,70.139106,5.382112,36.444232,-0.057096,0.128060,91.740851
131,70,1,2,1,1,0,0,0,0,0,...,8.911068,30.191045,30.926762,-20.991359,92.048538,6.191501,32.204819,-0.297133,0.145292,125.651157
132,48,1,0,1,0,0,0,0,0,1,...,8.559330,41.818329,43.318590,-10.066038,88.848639,5.685918,43.270040,-0.747640,0.153249,123.523670
133,77,0,0,1,0,0,0,0,0,0,...,7.122321,45.526514,46.416893,-1.766627,82.068654,4.714840,46.473165,-0.649283,0.167273,87.091593
134,71,1,2,1,1,0,0,0,0,0,...,7.859698,21.797408,22.808667,-17.244511,64.677609,5.393617,23.971309,-0.538387,0.119029,99.496646
135,30,1,0,1,0,1,0,0,1,0,...,6.422777,42.424447,42.377630,12.020920,73.894130,4.349602,43.226411,0.163147,0.161630,68.688869
136,70,1,0,1,0,0,0,0,0,0,...,5.480398,44.968895,45.722234,15.337966,63.758275,3.646514,45.532263,-0.205071,0.168967,50.985450
137,68,0,0,1,0,0,0,0,0,0,...,5.851199,52.722663,53.367068,33.955974,38.857387,4.144875,53.226413,-0.186106,0.098732,53.371855
138,67,0,0,1,1,0,0,0,0,0,...,5.679315,55.246152,55.659295,23.733982,72.429165,3.713243,55.754847,-0.105839,0.181474,56.465736
139,46,1,0,1,0,1,0,0,0,1,...,7.850145,51.005391,52.377313,12.032119,67.665868,5.442991,51.955631,-0.633099,0.127448,97.837712


In [106]:
train_missing = train_df.isnull().sum() * 100 / len(train_df)
train_missing = train_missing[train_missing != 0]
train_missing = pd.DataFrame({'missing percent': train_missing})
train_missing = train_missing.sort_values('missing percent', ascending=False)
display(train_missing.transpose().style.format(precision = 3))
for col in train_missing.index:
    print(col,train_df[col].unique())
    
test1_missing = test1_df.isnull().sum() * 100 / len(test1_df)
test1_missing = test1_missing[test1_missing != 0]
test1_missing = pd.DataFrame({'missing percent': test1_missing})
test1_missing = test1_missing.sort_values('missing percent', ascending=False)
display(test1_missing.transpose().style.format(precision = 3))
for col in test1_missing.index:
    print(col,test1_df[col].unique())
    
test2_missing = test2_df.isnull().sum() * 100 / len(test2_df)
test2_missing = test2_missing[test2_missing != 0]
test2_missing = pd.DataFrame({'missing percent': test2_missing})
test2_missing = test2_missing.sort_values('missing percent', ascending=False)
display(test2_missing.transpose().style.format(precision = 3))
for col in test2_missing.index:
    print(col,test2_df[col].unique())

missing percent


,target
missing percent,100.000


target [nan]


,target
missing percent,100.000


target [nan]


In [ ]:
# Pipeline([
#     ('imputer', SimpleImputer(strategy="median")),
#     ('std_scaler', StandardScaler()),
# ])